<a href="https://colab.research.google.com/github/Pavitra-khare/DA6401_ASS_2/blob/main/DA6401_ASS_2A_SWEEPs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login(key='b32b3fa2b8b3c86c7a8a232ec7d97bf3eda57182')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import wandb
from types import SimpleNamespace
import random



In [ ]:
# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
def img_size(img_w, filter_size, padding, stride):
    return (1 / 2) * (1 + (img_w - filter_size + (2 * padding)) / stride)

In [ ]:



class ConvNet(nn.Module):
    def __init__(self, in_channels=3, num_filters=[32, 64, 128, 256, 512], filter_size=[3, 3, 5, 5, 7],
                 activation=nn.ReLU(), stride=1, padding=1, pool_size=(2, 2), fc_size=512, num_classes=10,
                 dropout=0, batch_norm = 'Yes'):
        super(ConvNet, self).__init__()
        self.channels = in_channels
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.activation = activation
        self.stride = stride
        self.padding = padding
        self.pool_size = pool_size
        self.fc_size = fc_size
        self.num_classes = num_classes
        self.dropout = dropout
        self.batch_norm = batch_norm

        # Define convolutional layers
        self.conv1 = nn.Conv2d(self.channels, self.num_filters[0], self.filter_size[0], stride=self.stride, padding=self.padding)
        self.dropout1 = nn.Dropout2d(self.dropout)
        self.conv2 = nn.Conv2d(self.num_filters[0], self.num_filters[1], self.filter_size[1], stride=self.stride,padding=self.padding)
        self.dropout2 = nn.Dropout2d(self.dropout)
        self.conv3 = nn.Conv2d(self.num_filters[1], self.num_filters[2], self.filter_size[2], stride=self.stride,padding=self.padding)
        self.dropout3 = nn.Dropout2d(self.dropout)
        self.conv4 = nn.Conv2d(self.num_filters[2], self.num_filters[3], self.filter_size[3], stride=self.stride,padding=self.padding)
        self.dropout4 = nn.Dropout2d(self.dropout)
        self.conv5 = nn.Conv2d(self.num_filters[3], self.num_filters[4], self.filter_size[4], stride=self.stride,padding=self.padding)
        self.dropout5 = nn.Dropout2d(self.dropout)

        # Define batch normalization layers
        self.batchnorm1 = nn.BatchNorm2d(self.num_filters[0])
        self.batchnorm2 = nn.BatchNorm2d(self.num_filters[1])
        self.batchnorm3 = nn.BatchNorm2d(self.num_filters[2])
        self.batchnorm4 = nn.BatchNorm2d(self.num_filters[3])
        self.batchnorm5 = nn.BatchNorm2d(self.num_filters[4])

        # Define activation function
        self.activation = activation

        # Define max pooling layers
        self.pool = nn.MaxPool2d(self.pool_size, stride=2)  # for maxpool default stride is 2 and padding is 0


        # Calculating image width and height after each layer
        nxt_size1 = img_size(224, self.filter_size[0], self.padding, self.stride)
        nxt_size2 = img_size(nxt_size1, self.filter_size[1], self.padding, self.stride)
        nxt_size3 = img_size(nxt_size2, self.filter_size[2], self.padding, self.stride)
        nxt_size4 = img_size(nxt_size3, self.filter_size[3], self.padding, self.stride)
        nxt_size5 = img_size(nxt_size4, self.filter_size[4], self.padding, self.stride)
        nxt_size5 = int(nxt_size5)

        # Define dropout layer
        self.dropout_layer = nn.Dropout1d(self.dropout)

        # Define fully connected layer
        self.fc = nn.Linear(self.num_filters[4] * (nxt_size5 ** 2), self.fc_size)
        self.fc_bn = nn.BatchNorm1d(self.fc_size)  # Batch normalization for fully connected layer

        # Output layer
        self.output_layer = nn.Linear(self.fc_size, self.num_classes)

    # Forward propagation function
    def forward(self, x):
        # 1st conv layer
        x = self.conv1(x)
        if self.batch_norm == 'Yes':
            x = self.batchnorm1(x)
        else:
            x = x
        x = self.activation(x)
        x = self.pool(x)
        x = self.dropout1(x)


        # 2nd conv layer
        x = self.conv2(x)
        if self.batch_norm == 'Yes':
            x = self.batchnorm2(x)
        else:
            x = x
        # x = self.batchnorm2(x)
        x = self.activation(x)
        x = self.pool(x)
        x = self.dropout2(x)


        # 3rd conv layer
        x = self.conv3(x)
        if self.batch_norm == 'Yes':
            x = self.batchnorm3(x)
        else:
            x = x
        x = self.activation(x)
        x = self.pool(x)
        x = self.dropout3(x)


        # 4th conv layer
        x = self.conv4(x)
        if self.batch_norm == 'Yes':
            x = self.batchnorm4(x)
        else:
            x = x
        x = self.activation(x)
        x = self.pool(x)
        x = self.dropout4(x)

        # 5th conv layer
        x = self.conv5(x)
        if self.batch_norm == 'Yes':
            x = self.batchnorm5(x)
        else:
            x = x
        x = self.activation(x)
        x = self.pool(x)
        x = self.dropout5(x)

        # Flatten the output for the fully connected layer
        x = x.reshape(x.size(0), -1)
#         x = x.view(x.size(0), -1)

        # Fully connected layers
        x = self.fc(x)
        # Batch normalization before activation
        if self.batch_norm == 'Yes':
            x = self.fc_bn(x)
        else:
            x = x
        # x = self.fc_bn(x)
        x = self.activation(x)
        # Apply dropout
        x = self.dropout_layer(x)
        # Output layer
        x = self.output_layer(x)

        return x

# Making object of the ConvNet class
model = ConvNet().to(device)
print(model)


In [ ]:
 # Function to test the model
def test_on_valid_data(model, test_data):
    model.eval()  # Set the model to evaluation mode
    correct_val_label = 0
    total_val_label = 0
    with torch.no_grad():  # Turn off gradient calculation for validation
        for img, label in test_data:
            img, label = img.to(device), label.to(device)
            output = model(img)
            _, pred = torch.max(output, 1)
            correct_val_label += (pred == label).sum().item()
            total_val_label += label.size(0)

    # Calculate validation accuracy
    valid_accuracy = 100 * correct_val_label / total_val_label
    return valid_accuracy

In [ ]:
def model_train_val(model, train_data, val_data, epochs):
    best_val_acc = 0
    patience = 2
    counter = 0

    for epoch in range(epochs):
        model, avg_loss, train_accuracy = train_on_train_data(model, train_data)
        val_accuracy = test_on_valid_data(model, val_data)

        wandb.log({
            'Train loss': avg_loss,
            'Train accuracy': train_accuracy,
            'val_accuracy': val_accuracy,
            'epoch': epoch
        })

        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {avg_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%')

        # Early stopping logic
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break


In [ ]:
# Function to train the model on train data
def train_on_train_data(model, train_data):
    criterion = nn.CrossEntropyLoss()
    # optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model.train()  # Set the model to training mode
    training_loss = 0.0
    correct_train_label = 0
    total_train = 0
    # Training loop
    for inputs, labels in train_data:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        training_loss += loss.item()

        # Calculate training accuracy
        _, pred = torch.max(outputs, 1)
        correct_train_label += (pred == labels).sum().item()
        total_train += labels.size(0)

    # Calculate training loss and accuracy
    avg_loss = training_loss / len(train_data)
    train_accuracy = 100 * correct_train_label / total_train
    return model, avg_loss, train_accuracy



In [ ]:
# Training data loader function

train_data_dir = '/kaggle/input/my-dataset/inaturalist_12K/train'

def data_load(train_data_dir,data_augumentation):

    # Define variables to transeform the images to tensor and for data augumentation
    resize = transforms.Resize((224, 224))
    convert_to_tensor = transforms.ToTensor()
    normalize = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    resize_crop = transforms.RandomResizedCrop(224)
    h_flip = transforms.RandomHorizontalFlip()
    color_jitter = transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
    rand_rotation = transforms.RandomRotation(20)

    # Data augumentation
    if data_augumentation == 'Yes':
        transform_img = transforms.Compose([resize_crop,h_flip,color_jitter, rand_rotation, convert_to_tensor,normalize]) # Data transformations

    else:
        transform_img = transforms.Compose([resize,convert_to_tensor, normalize ])  # Data transformations

    # Load the dataset using ImageFolder and apply transformations
    training_data = ImageFolder(root=train_data_dir, transform=transform_img)
    # Splitting train dataset into training and validation indices
    train_index, val_index = train_test_split(list(range(len(training_data))), test_size=0.2, random_state=42)
    # Create DataLoader instances for training and validation sets
    random_train_sample = SubsetRandomSampler(train_index)
    train_data = DataLoader(
    training_data,
    batch_size=32,
    sampler=random_train_sample,
    num_workers=4,  # Add workers for parallel loading
    pin_memory=True  # Faster data transfer to GPU
    )

    random_val_sample = SubsetRandomSampler(val_index)
    validation_data = DataLoader(
    training_data,
    batch_size=32,
    sampler=random_val_sample,
    num_workers=4,  # Add workers for parallel loading
    pin_memory=True  # Faster data transfer to GPU
    )
    return train_data, validation_data



In [ ]:
# Test data loader function
test_data_dir = '/kaggle/input/my-dataset/inaturalist_12K/val'

def test_data_load(test_data_dir,data_augumentation):
    # Define variables to transeform the images to tensor and for data augumentation
    resize = transforms.Resize((224, 224))
    convert_to_tensor = transforms.ToTensor()
    normalize = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    resize_crop = transforms.RandomResizedCrop(224)
    h_flip = transforms.RandomHorizontalFlip()
    color_jitter = transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
    rand_rotation = transforms.RandomRotation(20)

    # Data augumentation
    if data_augumentation == 'Yes':
        transform_img = transforms.Compose([resize_crop,h_flip,color_jitter, rand_rotation, convert_to_tensor,normalize]) # Data transformations

    else:
        transform_img = transforms.Compose([resize,convert_to_tensor, normalize ])  # Data transformations

    # Load the dataset using ImageFolder and apply transformations
    test_data = ImageFolder(root=test_data_dir, transform=transform_img)
    # Apply DataLoader to load the test data with batch sizes
    testData = DataLoader(
    test_data,
    batch_size=32,
    num_workers=4,  # Add workers for parallel loading
    pin_memory=True  # Faster data transfer to GPU
    )

    return testData

In [ ]:
sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'kernel_size':{
            'values': [[3,3,3,3,3],[3,5,5,7,7],[3,5,3,5,7],[5,5,5,5,5]]#,[7,7,7,7,7]]
        },
        'dropout': {
            'values': [0.3, 0.2]
        },
        'activation': {
            'values': [ 'relu','mish','silu', 'gelu',]
        },
        'num_dense':{
            'values': [128, 256]
        },
        'batch_norm':{
            'values': ['Yes','No']
        },
        'filter_org':{
            'values': [[128,128,64,64,32],[32,64,128,256,512],[32,32,32,32,32],[32,64,64,128,128]]
        },
        'data_aug': {
            'values': ['No', 'Yes']
        }
    }
}

sweep_id = wandb.sweep(sweep_config,project="DL_ASS2")


In [ ]:
def main():
    '''
    WandB calls main function each time with differnet combination.

    We can retrive the same and use the same values for our hypermeters.

    '''

    with wandb.init() as run:
        run_name="ks"+str(wandb.config.kernel_size)+"ac-"+(wandb.config.activation)+"_drop-"+str(wandb.config.dropout)+"_daug-"+str(wandb.config.data_aug)+"_fs-"+str(wandb.config.filter_org)+"_bn-"+str(wandb.config.batch_norm)+"_dence-"+str(wandb.config.num_dense)
        wandb.run.name=run_name

        if  wandb.config.activation == 'relu':
            activ=nn.ReLU()
        elif wandb.config.activation == 'gelu':
            activ=nn.GELU()
        elif wandb.config.activation == 'silu':
            activ=nn.SiLU()
        elif wandb.config.activation == 'mish':
            activ=nn.Mish()

        model = ConvNet(in_channels=3, num_filters=wandb.config.filter_org, filter_size=wandb.config.kernel_size, activation=activ, stride=1,
                        padding=1, pool_size=(2,2), fc_size=wandb.config.num_dense, num_classes=10,dropout = wandb.config.dropout,batch_norm=wandb.config.batch_norm).to(device)

#         data_dir = '/kaggle/input/nature-12k/inaturalist_12K/train'
        train_data_dir = '/kaggle/input/my-dataset/inaturalist_12K/train'
        train, validation = data_load(train_data_dir,data_augumentation= wandb.config.data_aug)

        model_train_val(model, train, validation, epochs = 7)

#         model_train(model,train,validation)

wandb.agent("gvh886k1", function= main,count= 70) # calls main function for count number of times.
wandb.finish()